In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose
import pandas as pd
from tqdm import tqdm

In [2]:
def readInput(subset = 'train'):
    aortaDat = pd.read_csv(f'data/aortaP_{subset}_data.csv')
    aortaDat.rename({'Unnamed: 0': 'id'}, axis=1, inplace=True)
    brachDat = pd.read_csv(f'data/brachP_{subset}_data.csv')
    brachDat.rename({'Unnamed: 0': 'id'}, axis=1, inplace=True)

    if subset == 'train':
        targets = aortaDat[['id', 'target']]
        aortaDat = aortaDat.drop(['target'], axis=1)
        brachDat = brachDat.drop(['target'], axis=1)
    else:
        targets = None

    aortaDatLong = pd.melt(aortaDat, id_vars=['id'], var_name='time', value_name='aorta')
    brachDatLong = pd.melt(brachDat, id_vars=['id'], var_name='time', value_name='brach')

    aortaDatLong['time'] = aortaDatLong['time'].str.extract('(\d+)').astype(int)
    brachDatLong['time'] = brachDatLong['time'].str.extract('(\d+)').astype(int)

    merge_df = (
        brachDatLong
            .set_index(['id', 'time'])
            .join(aortaDatLong.set_index(['id', 'time']), how='outer', validate='one_to_one')
            .reset_index()
    )

    return (merge_df, targets)

In [3]:
all_train, all_targets = readInput('train')

In [4]:
generator1 = torch.Generator().manual_seed(42)
train_ids, val_ids = random_split(all_train.id.unique(), [0.7, 0.3], generator=generator1)

In [5]:
def data_load(all_data, subset):
    data = []

    for idx in subset:
        subset_dat = all_data[all_data['id'] == idx].copy()

        subset_dat.brach.interpolate(method='linear', inplace=True)
        subset_dat.aorta.interpolate(method='linear', inplace=True)

        subset_dat.aorta.bfill(inplace=True)
        subset_dat.brach.bfill(inplace=True)

        subset_dat.aorta.ffill(inplace=True)
        subset_dat.brach.ffill(inplace=True)

        data.append({
            'id': idx,
            'data': torch.transpose(torch.tensor(subset_dat[['brach', 'aorta']].values, dtype=torch.float32), 0, 1),
            'target': torch.tensor(all_targets[all_targets['id'] == idx]['target'].values[0], dtype=torch.long)
        })

    return data

In [6]:
class TSDataLoader(Dataset):
    def __init__(self, subset, transform=None):
        self.data = subset
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = (self.data[idx]['data'] - 100) / 40

        if self.transform:
            X = self.transform(X)

        if 'target' in self.data[idx]:
            return (X, self.data[idx]['target'])
        return X

In [7]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift

    def __call__(self, sample):
        shift = (torch.rand((1,1)) - 0.5) * 2
        return sample + shift
    
class RandomSubsample(object):
    def __init__(self, subsample):
        self.subsample = subsample

    def __call__(self, sample):
        idx = sorted(torch.randperm(sample.shape[1])[:100].numpy())
        return sample[:, idx]

In [8]:
training_data = TSDataLoader(data_load(all_train, train_ids), transform=Compose([RandomShift(0.3), RandomSubsample(75)]))
test_data = TSDataLoader(data_load(all_train, val_ids), transform=RandomSubsample(100))

In [9]:
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

torch.device(device)
print(f"Using {device} device")

Using mps device


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.LazyConv1d(16, 3),
            nn.MaxPool1d(kernel_size=2),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyConv1d(32, 5),
            nn.MaxPool1d(kernel_size=2),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyConv1d(64, 5),
            nn.MaxPool1d(kernel_size=2),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(num_classes)
        )


    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = NeuralNetwork(6)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

/Users/ajb5d/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [13]:
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): LazyConv1d(0, 16, kernel_size=(3,), stride=(1,))
    (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): LazyConv1d(0, 32, kernel_size=(5,), stride=(1,))
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): LazyConv1d(0, 64, kernel_size=(5,), stride=(1,))
    (9): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): LazyLinear(in_features=0, out_features=64, bias=True)
    (14): ReLU()
    (15): LazyLinear(in_features=0, out_features=6, bias=True)
  )
)

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [15]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.822543  [    0/ 2450]
loss: 1.778925  [ 1600/ 2450]
Test Error: 
 Accuracy: 19.0%, Avg loss: 1.789219 

Epoch 2
-------------------------------
loss: 1.781324  [    0/ 2450]
loss: 1.772575  [ 1600/ 2450]
Test Error: 
 Accuracy: 23.4%, Avg loss: 1.769860 

Epoch 3
-------------------------------
loss: 1.760284  [    0/ 2450]
loss: 1.676159  [ 1600/ 2450]
Test Error: 
 Accuracy: 22.7%, Avg loss: 1.762182 

Epoch 4
-------------------------------
loss: 1.771513  [    0/ 2450]
loss: 1.786898  [ 1600/ 2450]
Test Error: 
 Accuracy: 21.7%, Avg loss: 1.766655 

Epoch 5
-------------------------------
loss: 1.758940  [    0/ 2450]
loss: 1.732722  [ 1600/ 2450]
Test Error: 
 Accuracy: 25.3%, Avg loss: 1.749240 

Epoch 6
-------------------------------
loss: 1.749501  [    0/ 2450]
loss: 1.662830  [ 1600/ 2450]
Test Error: 
 Accuracy: 25.4%, Avg loss: 1.730559 

Epoch 7
-------------------------------
loss: 1.738891  [    0/ 2450]
loss: 1.607929  [ 